In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import joblib
import pickle

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)


In [45]:
data=pd.read_csv('data/df_KOI_NOT_Capped.csv')
data.head(5)

,koi_period,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_steff,koi_slogg,koi_srad,koi_kepmag,koi_disposition
0,9.488036,2.95750,615.8,2.26,793.0,93.59,35.8,0,0,0,0,5455.0,4.467,0.927,15.347,CONFIRMED
1,54.418383,4.50700,874.8,2.83,443.0,9.11,25.8,0,0,0,0,5455.0,4.467,0.927,15.347,CONFIRMED
2,19.899140,1.78220,10829.0,14.60,638.0,39.30,76.3,0,0,0,0,5853.0,4.544,0.868,15.436,CANDIDATE
3,1.736952,2.40641,8079.2,33.46,1395.0,891.96,505.6,0,1,0,0,5805.0,4.564,0.791,15.597,FALSE POSITIVE
4,2.525592,1.65450,603.3,2.75,1406.0,926.16,40.9,0,0,0,0,6031.0,4.438,1.046,15.509,CONFIRMED


In [35]:
data['koi_disposition'] = data['koi_disposition'].map({
    'FALSE POSITIVE': 0,
    'CANDIDATE': 1,
    'CONFIRMED': 2
})


In [36]:
X = data.drop('koi_disposition', axis=1)
y=data['koi_disposition']



In [37]:
data.describe()

,koi_period,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_steff,koi_slogg,koi_srad,koi_kepmag,koi_disposition
count,9564.000000,9564.000000,9.564000e+03,9564.000000,9564.000000,9.564000e+03,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000
mean,75.671358,5.621606,2.290432e+04,99.077250,1077.514534,7.490517e+03,250.903691,0.208595,0.232748,0.197512,0.120033,5709.107277,4.315009,1.701053,14.264633,0.781159
std,1334.744046,6.471554,8.079020e+04,3018.723391,840.875323,1.565158e+05,781.868222,4.767290,0.422605,0.398142,0.325018,781.672343,0.425019,6.011383,1.385378,0.863287
min,0.241843,0.052000,0.000000e+00,0.080000,25.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,2661.000000,0.047000,0.109000,6.966000,0.000000
25%,2.733684,2.437750,1.668000e+02,1.430000,553.000000,2.216000e+01,12.300000,0.000000,0.000000,0.000000,0.000000,5333.000000,4.232750,0.835750,13.440000,0.000000
50%,9.752831,3.792600,4.211000e+02,2.390000,878.000000,1.416000e+02,23.000000,0.000000,0.000000,0.000000,0.000000,5767.000000,4.438000,1.000000,14.520000,0.000000
75%,40.715178,6.276500,1.341775e+03,13.112500,1352.500000,8.067975e+02,71.125000,0.000000,0.000000,0.000000,0.000000,6099.000000,4.539000,1.313000,15.322000,2.000000
max,129995.778400,138.540000,1.541400e+06,200346.000000,14667.000000,1.094755e+07,9054.700000,465.000000,1.000000,1.000000,1.000000,15896.000000,5.364000,229.908000,20.003000,2.000000


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

# 3. Transform test data (DON'T fit again!)
X_test_scaled = scaler.transform(X_test)




pickle.dump(scaler, open("scaler.pkl", "wb"))

In [40]:

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

In [41]:
from xgboost import XGBClassifier  # <-- Step 1: Import

# === Define Models & Hyperparameters ===
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)  # <-- Step 2: Add model
}

params = {
    "Random Forest": {
        'n_estimators': [100, 200],
        'max_depth': [None, 20],
        'max_features': ['sqrt', 'log2']
    },

    "Decision Tree": {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    },

    "Gradient Boosting": {
        'n_estimators': [100, 200],
        'learning_rate': [0.1, 0.05],
        'max_depth': [3, 5]
    },

    "AdaBoost": {
        'n_estimators': [50, 100],
        'learning_rate': [0.1, 1.0]
    },

    "Logistic Regression": {
        'C': [0.1, 1.0, 10.0],
        'solver': ['lbfgs', 'liblinear']
    },

    "XGBoost": {  # <-- Step 3: Add hyperparameters
        'n_estimators': [100, 200],
        'learning_rate': [0.1, 0.05],
        'max_depth': [3, 5, 7]
    }
}

# === Train and Evaluate ===
best_model = None
best_score = 0
model_scores = {}

for name, model in models.items():
    print(f"\n🔧 Training {name}...")
    grid = GridSearchCV(model, params[name], cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)

    y_pred = grid.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f"✅ {name} Results:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"Best Params: {grid.best_params_}")
    
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    model_scores[name] = {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1_score': f1,
        'best_params': grid.best_params_
    }

    if acc > best_score:
        best_score = acc
        best_model = grid.best_estimator_

# === Save Best Model ===
print(f"\n🏆 Best Model: {type(best_model).__name__} with Accuracy: {best_score:.4f}")
pickle.dump(best_model, open("best_model.pkl", "wb"))



🔧 Training Random Forest...
✅ Random Forest Results:
Accuracy:  0.9080
Precision: 0.8831
Recall:    0.8780
F1-Score:  0.8802
Best Params: {'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 200}
[[922  14   3]
 [ 14 309  82]
 [  7  56 506]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       939
           1       0.82      0.76      0.79       405
           2       0.86      0.89      0.87       569

    accuracy                           0.91      1913
   macro avg       0.88      0.88      0.88      1913
weighted avg       0.91      0.91      0.91      1913


🔧 Training Decision Tree...
✅ Decision Tree Results:
Accuracy:  0.8949
Precision: 0.8643
Recall:    0.8589
F1-Score:  0.8607
Best Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 5}
[[922  14   3]
 [  9 290 106]
 [  4  65 500]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       939
           1

/home/muhammad-ilyas-khan/anaconda3/envs/myenv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/home/muhammad-ilyas-khan/anaconda3/envs/myenv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/home/muhammad-ilyas-khan/anaconda3/envs/myenv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classificatio

✅ Logistic Regression Results:
Accuracy:  0.6357
Precision: 0.5564
Recall:    0.5578
F1-Score:  0.5306
Best Params: {'C': 10.0, 'solver': 'lbfgs'}
[[723  73 143]
 [144  52 209]
 [115  13 441]]
              precision    recall  f1-score   support

           0       0.74      0.77      0.75       939
           1       0.38      0.13      0.19       405
           2       0.56      0.78      0.65       569

    accuracy                           0.64      1913
   macro avg       0.56      0.56      0.53      1913
weighted avg       0.61      0.64      0.60      1913


🔧 Training AdaBoost...
✅ AdaBoost Results:
Accuracy:  0.8813
Precision: 0.8480
Recall:    0.8410
F1-Score:  0.8427
Best Params: {'learning_rate': 1.0, 'n_estimators': 100}
[[917  13   9]
 [  2 274 129]
 [  5  69 495]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       939
           1       0.77      0.68      0.72       405
           2       0.78      0.87      0.82 

/home/muhammad-ilyas-khan/anaconda3/envs/myenv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:10:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/muhammad-ilyas-khan/anaconda3/envs/myenv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:10:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/muhammad-ilyas-khan/anaconda3/envs/myenv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:10:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/muhammad-ilyas-khan/anaconda3/envs/myenv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:10:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



✅ XGBoost Results:
Accuracy:  0.9090
Precision: 0.8826
Recall:    0.8799
F1-Score:  0.8809
Best Params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
[[921  16   2]
 [  7 311  87]
 [  5  57 507]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       939
           1       0.81      0.77      0.79       405
           2       0.85      0.89      0.87       569

    accuracy                           0.91      1913
   macro avg       0.88      0.88      0.88      1913
weighted avg       0.91      0.91      0.91      1913


🏆 Best Model: GradientBoostingClassifier with Accuracy: 0.9106


In [42]:
y_pred

array([0, 1, 2, ..., 2, 1, 0], shape=(1913,))

In [43]:
inp=scaler.transform(data.head(2).drop('koi_disposition', axis=1))
result=best_model.predict(inp)
result[0]


/home/muhammad-ilyas-khan/anaconda3/envs/myenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


np.int64(1)